# Úkol č. 1 - vizualizace dat a web scraping (do 20. října)

  * V rámci tohoto úkolu musíte stáhnout dat z webu (tzv. _web scraping_, velmi základní) a následně data zpracovat a vizualizovat.
  * Cílem bude stáhnout data ze serveru https://www.volby.cz týkající se voleb do zastupitelstva Vámi vybraného (většího) města, uložit tato data v tabulkovém formátu a pak vymyslet vizualizace a zobrazení dat, které umožní orientaci v těchto datech a zvýrazní zajímavé informace a zobrazit přehledně časový vývoj různých veličin.
 
> **Úkoly jsou zadány tak, aby Vám daly prostor pro invenci. Vymyslet _jak přesně_ budete úkol řešit, je důležitou součástí zadání a originalita či nápaditost bude také hodnocena!**

## Výběr zdroje dat

Vyberte si větší město, které má zastupitelstvo druhu 3 (Zastupitelstvo statutárního města) a strojově stáhněte informace o stranách a kandidátkách z následujících let:
 * [2002](https://www.volby.cz/pls/kv2002/kv12?xjazyk=CZ&xid=0), [2006](https://www.volby.cz/pls/kv2006/kv12?xjazyk=CZ&xid=0), [2010](https://www.volby.cz/pls/kv2010/kv12?xjazyk=CZ&xid=0), [2014](https://www.volby.cz/pls/kv2014/kv12?xjazyk=CZ&xid=0) a [2018](https://www.volby.cz/pls/kv2018/kv12?xjazyk=CZ&xid=0).
 

## Pokyny k vypracování

**Základní body zadání**, za jejichž (poctivé) vypracování získáte **8 bodů**:
  * Strojově stáhněte data pro vybrané město a uložte je všechny do (asi dvou) přehledných tabulek ve formátu _csv_.
  * Data musí obsahovat _alespoň_ toto:
    * Vývoj výsledků (v procentech i počtu hlasů) pro jednotlivé strany v jednotlivých letech.
    * Seznam všech kandidátů všech stran v jednotlivých letech, u kandidáta by mělo být zaznamenáno: jméno, věk v době voleb, navrhující strana, politická příslušnost, volební zisk (procento i počet hlasů), pořadí na kandidátce, pořadí zvolení, jestli získal mandát (tyto informace získáte souhrnně ve _jmenných seznamech_).
  * V druhé části Vašeho Jupyter notebooku pracujte s těmito tabulkami načtenými z _csv_ souboru (aby opravující nemusel spouštět stahování z webu).
  * Tabulky ve formátu _csv_ také odevzdejte.
  * S využitím vybraných nástrojů zpracujte data a vymyslete vizualizace a grafy, aby bylo vidět následující:
    * Časový vývoj (po rocích voleb) počtu kandidujících stran i lidí a to celkově i po jednotlivých stranách (ve volbách, kterých se daná strana účastnila).
    * Věkovou strukturu kandidátů celkově i za jednotlivé strany a vývoj této struktury během jednotlivých voleb.
    * Časový vývoj volební účasti a volebních výsledků jednotlivých stran.
    * Časový vývoj podílu kandidujících s titulem a bez titulu.

**Další body zadání** za případné další body (můžete si vybrat, maximum bodů za úkol je každopádně 12 bodů):
  * (až +2 body) U titulů se pokuste rozlišit i různé stupně vzdělání: bakalářský, magisterský, doktorský a vyšší, vojenská hodnost atp. Zkuste odhadnout i podíl žen na kandidátkách.
  * (až +4 body) Pokuste se u jednotlivých kandidátů zjistit, zda kandidovali ve více volbách. Najděte 10 nejpilnějších kandidátů a vypište jejich volební zisky a za jaké strany kandidovali.
  * (až +2 body) Najděte nějaký balíček, který Vám dovolí do Vašeho notebooku zavést interaktivní prvky, např. si vyberete v select-boxu stranu a Váš notebook zobrazí grafy pouze pro ni atp.

## Poznámky k odevzdání

  * Řiďte se pokyny ze stránky https://courses.fit.cvut.cz/BI-VZD/homeworks/index.html.
  * Odevzdejte nejen Jupyter Notebook, ale i _csv_ soubor(y) se staženými daty.
  * Opravující Vám může umožnit úkol dodělat či opravit a získat tak další body. První verze je ale důležitá a bude-li odbytá, budete za to penalizováni.

In [1]:
import numpy as np
import pandas as pd
import sklearn as skit
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [78]:
party_data = []
people_data = []

years = [2006, 2010, 2014, 2018]

for year in years:
    '''
        reads data from volby.cz 
    '''
    url = 'https://www.volby.cz/pls/kv' + str(year) + '/kv1111?xjazyk=CZ&xid=1&xdz=3&xnumnuts=5103&xobec=563889&xstat=0&xvyber=0'
    dfs = pd.read_html(url,flavor='html5lib')
    party_data.append(dfs[1])
    url = 'https://www.volby.cz/pls/kv' + str(year) + '/kv21111?xjazyk=CZ&xid=1&xv=11&xdz=3&xnumnuts=5103&xobec=563889&xstrana=0'
    dfs = pd.read_html(url,flavor='html5lib')
    #dfs[0].columns = dfs[0].comlumns.to_frame()[1]
    people_data.append(dfs[0])
    
# changes header from 2 levels to 1 level
for df in party_data:
    df.columns = df.columns.to_frame()[1]
for df in people_data:
    df.columns = df.columns.to_frame()[1]

In [118]:
people_data[1]
party_data[2]

1,číslo,název,abs.,v %,Početkandidátů,Přepočtenýzákladdle počtukandidátů,Přepočtené %platných hlasů,Početmandátů,Podílyhlasů
0,1,Nový směr-společná kandidátka,28 259,271,39,"1 042 675,00",271,0,-
1,2,Česká str.sociálně demokrat.,102 429,982,39,"1 042 675,00",982,4,X
2,3,Starostové pro Liberecký kraj,172 933,1659,39,"1 042 675,00",1658,8,X
3,4,Změna pro Liberec,265 450,2546,39,"1 042 675,00",2545,12,X
4,5,Úsvit přímé demokracie,14 479,139,39,"1 042 675,00",138,0,-
5,6,KDU-ČSL a nezávislí,23 964,230,39,"1 042 675,00",229,0,-
6,7,Str.pro otevřenou společnost,6 044,58,36,"962 469,23",62,0,-
7,8,TOP 09,56 952,546,39,"1 042 675,00",546,2,X
8,9,Liberec ještě žije,16 293,156,39,"1 042 675,00",156,0,-
9,10,Strana svobodných občanů,19 909,191,39,"1 042 675,00",190,0,-


In [110]:
# reads data for year 2002
url = 'http://www.volby.cz/pls/kv2002/kv21111?xjazyk=CZ&xid=1&xv=11&xdz=3&xnumnuts=5103&xobec=563889&xstrana=0'
people_data_2002 = pd.read_html(url, displayed_only=False)
people_data_2002[0].columns = people_data_2002[0].columns.to_frame()[1]

In [113]:
display(people_data_2002[0].columns)
display(people_data[0].columns)

Index(['Kandidátnílistina', 'Kandidátnílistina.1', 'Poř.číslo',
       'Příjmení, jméno', 'Tituly', 'Věk', 'Navrh.strana', 'Polit.přísl.',
       'abs.', 'v %', 'Pořadí', 'Mandát'],
      dtype='object', name=1)

Index(['číslo', 'název', 'poř.číslo', 'příjmení, jméno, tituly', 'věk',
       'Navrhujícístrana', 'Politickápříslušnost', 'abs.', 'v %',
       'Pořadízvolení/náhradníka', 'Mandát'],
      dtype='object', name=1)

In [123]:
# reads data for year 2002
url = 'http://www.volby.cz/pls/kv2002/kv1211?xjazyk=CZ&xid=1&xv=1&xdz=3&xnumnuts=5103'
party_data_2002 = pd.read_html(url, displayed_only=False)
party_data_2002[0].columns = party_data_2002[0].columns.to_frame()[1]
party_data_2002[1].columns = party_data_2002[1].columns.to_frame()[1]

In [124]:
people_data_2002[0]
party_data_2002[1]

1,Volební strana,abs.,v %,abs.,v %
0,Křesťanská a demokratická unie - Československ...,35 474,3.60,0,0.00
1,Česká strana sociálně demokratická,155 273,15.76,7,17.95
2,Komunistická strana Čech a Moravy,105 909,10.75,5,12.82
3,Občanská demokratická strana,273 081,27.72,13,33.33
4,NEZÁVISLÍ,50 781,5.15,2,5.13
5,"Pravý Blok-strana za ODVOLAT.politiků, NÍZKÉ d...",2 874,0.29,0,0.00
6,Unie svobody - Demokratická unie,53 994,5.48,2,5.13
7,Strana pro otevřenou společnost,97 869,9.93,4,10.26
8,Demokratická regionální strana,60 223,6.11,3,7.69
9,Hnutí nezávislých za harmonický rozvoj obcí a ...,19 386,1.97,0,0.00


In [59]:
pd.DataFrame(party_data).to_csv("party_data.csv")
pd.DataFrame(people_data).to_csv("people_data.csv")

In [125]:
pd.DataFrame(party_data[0]).info()
pd.DataFrame(people_data[0]).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 9 columns):
číslo                                 10 non-null int64
název                                 10 non-null object
abs.                                  10 non-null object
v %                                   10 non-null int64
Početkandidátů                        10 non-null int64
Přepočtenýzákladdle počtukandidátů    10 non-null object
Přepočtené %platných hlasů            10 non-null int64
Početmandátů                          10 non-null int64
Podílyhlasů                           10 non-null object
dtypes: int64(5), object(4)
memory usage: 800.0+ bytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287 entries, 0 to 286
Data columns (total 11 columns):
číslo                       287 non-null int64
název                       287 non-null object
poř.číslo                   287 non-null int64
příjmení, jméno, tituly     287 non-null object
věk                         287 non-null 